In [1]:
# Some path wizardry to make python acknowledge relative paths, just ignore this part...
import sys; sys.path.append('..')
import networkx as nx
import pandas as pd
from ipysigma import Sigma
from pelote import (
    graph_to_nodes_dataframe,
    table_to_bipartite_graph,
    monopartite_projection,
    graph_to_edges_dataframe,
    global_threshold_sparsification,
    multiscale_backbone,
    simmelian_backbone,
    SimmelianBackboneSparsifier,
    read_graphology_json
)
from pelote.graph import remove_edges

In [2]:
df = pd.read_csv('../data/bipartite2.csv')
df

,account,post
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
36806,4173,1978
36807,4174,1978
36808,3893,1978
36809,776,1979


In [3]:
g = table_to_bipartite_graph(df, 'account', 'post')

In [4]:
m = monopartite_projection(g, 'account', metric='cosine')

In [5]:
graph_to_edges_dataframe(m)['weight'].mean()

0.2712789108985405

In [6]:
remove_edges(m, lambda u, v, a: a['weight'] > 0.25)

In [7]:
graph_to_edges_dataframe(m)

,source,target,weight
0,0,2,0.500000
1,0,4,0.353553
2,0,5,0.500000
3,0,12,0.258199
4,0,27,0.577350
...,...,...,...
179547,6117,6118,1.000000
179548,6135,6137,1.000000
179549,6142,6144,0.707107
179550,6148,6150,1.000000


In [8]:
Sigma(
    m,
    node_size=m.degree,
    node_size_range=(3, 10),
    node_borders=True,
    default_node_border_color='black'
)

Sigma(nx.Graph with 4,175 nodes and 179,552 edges)

In [9]:
n = global_threshold_sparsification(m, 0.5, keep_connected=True)

In [10]:
Sigma(
    n,
    node_size=m.degree,
    node_size_range=(3, 10),
    node_borders=True,
    default_node_border_color='black'
)

Sigma(nx.Graph with 4,175 nodes and 70,756 edges)

In [11]:
o = multiscale_backbone(m,keep_connected=True)

In [12]:
Sigma(
    o,
    node_size=m.degree,
    node_size_range=(3, 10),
    node_borders=True,
    default_node_border_color='black'
)

Sigma(nx.Graph with 4,175 nodes and 17,178 edges)

In [13]:
sparsifier = SimmelianBackboneSparsifier(5, 2, keep_connected=True)
relevant = list(sparsifier.relevant_edges(m))

In [14]:
for u, v, a in m.edges.data():
    a['relevant'] = False

for edge in relevant:
    m.edges[edge[0], edge[1]]['relevant'] = True

for u, v, a in m.edges.data():
    if not a['relevant']:
        a['weight'] = 0

In [15]:
graph_to_edges_dataframe(m)

,source,target,weight,relevant
0,0,2,0.500000,True
1,0,4,0.000000,False
2,0,5,0.500000,True
3,0,12,0.000000,False
4,0,27,0.577350,True
...,...,...,...,...
179547,6117,6118,1.000000,True
179548,6135,6137,1.000000,True
179549,6142,6144,0.707107,True
179550,6148,6150,1.000000,True


In [16]:
Sigma(
    m,
    node_size=m.degree,
    node_size_range=(3, 10),
    edge_color='relevant',
    edge_color_palette={True:'black', False:'#ccc'},
    node_borders=True,
    default_node_border_color='black',
    edge_zindex=lambda u,v,a:  1 if a['relevant']==True else 0,
    edge_size_range=0.5,
    layout_settings={"weightedMass": True}
)

Sigma(nx.Graph with 4,175 nodes and 179,552 edges)

In [17]:
fb = read_graphology_json('../data/socfb.json')

In [18]:
Sigma(fb)

Sigma(nx.Graph with 769 nodes and 16,656 edges)

In [19]:
sparsifier = SimmelianBackboneSparsifier(10, 5, in_or_out_edge="out", reciprocity=True)
relevant = list(sparsifier.relevant_edges(fb))
for u, v, a in fb.edges.data():
    a['relevant'] = False

for edge in relevant:
    fb.edges[edge[0], edge[1]]['relevant'] = True

for u, v, a in fb.edges.data():
    if not a['relevant']:
        a['weight'] = 0

In [20]:
Sigma(
    fb,
    node_size=fb.degree,
    node_size_range=(3, 10),
    edge_color='relevant',
    edge_color_palette={True:'black', False:'#ccc'},
    node_borders=True,
    default_node_border_color='black',
    edge_zindex=lambda u,v,a:  1 if a['relevant']==True else 0,
    edge_size_range=0.5,
    layout_settings={"weightedMass": True}
)

Sigma(nx.Graph with 769 nodes and 16,656 edges)

In [21]:
sparse = sparsifier(fb)

In [22]:
Sigma(
    sparse,
    node_size=sparse.degree,
    node_size_range=(3, 10),
    edge_color='relevant',
    edge_color_palette={True:'black', False:'#ccc'},
    node_borders=True,
    default_node_border_color='black',
    edge_zindex=lambda u,v,a:  1 if a['relevant']==True else 0,
    edge_size_range=0.5,
    layout_settings={"weightedMass": True}
)

Sigma(nx.Graph with 769 nodes and 5,539 edges)